# Gradient Descent Implementation
---
Let's first use `sklearn` to predict housing prices based on the sample dataset using linear regression. Later, let's implement gradient descent and see if we come up with the same predicted housing prices.

In [1]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.spatial import distance
import sklearn
from scipy import stats
from sklearn.datasets.samples_generator import make_regression 

living_area = np.array([2104, 1600, 2400, 1416, 3000])
num_bedrooms = np.array([3, 3, 3, 2, 4])
price = np.array([400, 330, 369, 232, 540]) # we want to predict the price
x = np.concatenate((living_area, num_bedrooms), axis = 0)

# add in 1s so that we can use theta_0 when computing the dot product
x = np.concatenate((x, np.ones(len(num_bedrooms))), axis = 0)
x = x.reshape(3, 5).T

from sklearn import linear_model
clf = linear_model.LinearRegression()
clf.fit(x, price)

print "coefficients of the hypothesis: ", clf.coef_

for i, sample in enumerate(x):
    predicted_price = 0
    predicted = np.dot(sample, clf.coef_)
    print "predicted: $%f, actual: $%f" % (predicted, price[i])

coefficients of the hypothesis:  [  6.38433756e-02   1.03436047e+02   0.00000000e+00]
predicted: $444.634602, actual: $400.000000
predicted: $412.457541, actual: $330.000000
predicted: $463.532241, actual: $369.000000
predicted: $297.274313, actual: $232.000000
predicted: $605.274313, actual: $540.000000


## Batch Gradient Descent
---
Gradient descent can be implemented in two main ways: batch gradient descent or stochastic gradient descent. The update rule for gradient descent follows. Unfortunately, Github doesn't render $\LaTeX$'s `align` environment properly. You can see the [properly formatted derivations here](http://carsontang.github.io/assets/cs229_derivations.pdf).

$\begin{align}
\theta_j &:= \theta_j - \alpha \cdot \frac{\partial}{\partial \theta_j} J(\theta) \\
&= \theta_j - \alpha \cdot \sum_{i = 0}^m \left(h_{\theta}(x^{(i)}) - y^{(i)} \right) \cdot x_j^{(i)}\\
&= \theta_j - \alpha \cdot \sum_{i = 0}^m \left( \theta^T x^{(i)} - y^{(i)} \right) \cdot x_j^{(i)}
\end{align}$

The derivation of $\frac{\partial}{\partial \theta_j} J(\theta)$ follows

$\begin{align}
\frac{\partial}{\partial \theta_j} J(\theta) &=
\frac{\partial}{\partial \theta_j}\left( \frac{1}{2} \sum_{i = 0}^m \left( \theta^T x^{(i)} - y^{(i)} \right)^2 \right)\\
&= \frac{1}{2} \frac{\partial}{\partial \theta_j}
\left[
    \left( \theta^T x^{(0)} - y^{(0)} \right)^2 +
    \left( \theta^T x^{(1)} - y^{(1)} \right)^2 +
    \cdots +
    \left( \theta^T x^{(m)} - y^{(m)} \right)^2
\right]\\
&= \underbrace{\frac{1}{2}
\left[
    \frac{\partial}{\partial \theta_j} \left( \theta^T x^{(0)} - y^{(0)} \right)^2 +
    \frac{\partial}{\partial \theta_j} \left( \theta^T x^{(1)} - y^{(1)} \right)^2 +
    \cdots +
    \frac{\partial}{\partial \theta_j} \left( \theta^T x^{(m)} - y^{(m)} \right)^2
\right]}_{\text{via the sum rule}}\\
&= \frac{1}{2}
\left[
    2 \left( \theta^T x^{(0)} - y^{(0)} \right) \cdot \frac{\partial}{\partial \theta_j}
        \left( \theta^T x^{(0)} - y^{(0)} \right) +
    2 \left( \theta^T x^{(1)} - y^{(1)} \right) \cdot \frac{\partial}{\partial \theta_j}
        \left( \theta^T x^{(1)} - y^{(1)} \right) +
    \cdots +
    2 \left( \theta^T x^{(m)} - y^{(m)} \right) \cdot \frac{\partial}{\partial \theta_j}
        \left( \theta^T x^{(m)} - y^{(m)} \right)
\right]\\
&= \frac{1}{2}
\left[
    2 \left( \theta^T x^{(0)} - y^{(0)} \right) \cdot x^{(0)}_j +
    2 \left( \theta^T x^{(1)} - y^{(1)} \right) \cdot x^{(1)}_j +
    \cdots +
    2 \left( \theta^T x^{(m)} - y^{(m)} \right) \cdot x^{(m)}_j
\right]\\
&= \frac{1}{2} \underbrace{\sum_{i = 0}^m 2 \left( \theta^T x^{(i)} - y^{(i)} \right)
    \cdot x^{(i)}_j}_{\text{extract the 2 via the distributive property of multiplication}} \\
&= \sum_{i = 0}^m \left( \theta^T x^{(i)} - y^{(i)} \right) \cdot x^{(i)}_j
\end{align}$

Below is an implementation of batch gradient descent.

In [72]:
# X = training dataset, or input data that'll allow us to predict the target dataset (below)
# Y = target dataset, or variable we're trying to predict
# H = hypothesis, the function that'll predict Y

def partial_of_cost_wrt_j(Y, X, theta, H, i, j):
    return (Y[i] - H(X[i], theta)) * X[i][j]
        
def do_batch_gradient_descent(theta, X, Y, H, alpha):
    new_theta = [v for v in theta]
    for j in range(len(theta)): # for each feature in theta
        delta = sum([partial_of_cost_wrt_j(Y, X, theta, H, i, j) for (i, _) in enumerate(Y)])
        new_theta[j] += alpha * delta
    return new_theta

# least squares cost function
def cost(X, Y, H, theta):
    return 0.5 * sum([ (H(X[i], theta) - Y[i])**2 for (i, _) in enumerate(Y) ])
    
def fit(X, Y, learning_rate, descent_algorithm):
    def hypothesis(X_i, theta):
        return np.dot(X_i, theta)
    
    theta = np.ones(x.shape[1]) # initialize parameters to all be 1's
    c = cost(X, Y, hypothesis, theta)
    
    num_iterations = 0
    while c > 100:
        num_iterations += 1
#         print "cost: %f, theta: %s" % (cost(X, Y, hypothesis, theta), theta)
        theta = descent_algorithm(theta, X, Y, hypothesis, learning_rate)
        if abs(cost(X, Y, hypothesis, theta) - c) < 0.001:
            break
        c = cost(X, Y, hypothesis, theta)
    print "with learning_rate = %5s, converged after %5d iterations" % (learning_rate, num_iterations)
    return theta

coefficients = fit(x, price, 0.000000001, do_batch_gradient_descent)
coefficients = fit(x, price, 0.00000001, do_batch_gradient_descent)

print
print "coefficients of the hypothesis: %s" % coefficients
for i, sample in enumerate(x):
    predicted = np.dot(sample, coefficients)
    print "predicted: $%f, actual: $%f" % (predicted, price[i])

with learning_rate = 1e-09, converged after   412 iterations
with learning_rate = 1e-08, converged after    42 iterations

coefficients of the hypothesis: [0.17515718101812017, 0.99887165461797678, 0.99963822144941095]
predicted: $372.526962, actual: $400.000000
predicted: $284.247743, actual: $330.000000
predicted: $424.373488, actual: $369.000000
predicted: $251.019950, actual: $232.000000
predicted: $530.466668, actual: $540.000000


The `fit` function above uses **batch gradient descent (BGD)** to find the parameters $\theta$ that result in as low a **cost** as possible. `fit` stops when gradient descent **converges**. Convergence is determined by the programmer. In this case, I printed out the cost after every iteration of BGD, and I determined that it was not changing too much relatively speaking. One value that you need to pay attention to is the **learning rate**, $\alpha$. The BGD algorithm **may never converge if the learning rate is too high**. Set the learning rate to be 10x higher, and you'll notice that the cost will increase after every iteration. This means BGD is taking big strides that are making the algorithm miss a point that'll result in a lower cost. If you set the learning rate to be 10x lower, you'll notice BGD will require more iterations to converge (412 compared to 42).

The reason the learning rate needs to be so low is the features of the training set vary greatly in terms of value. The first feature is **living area**, which is typically in the thousands. The second features is **number of bedrooms**, which is usually a single digit number (and very rarely in the double digits, unless you're living in a giant mansion). Furthermore, the target values, the prices of the homes, are in the hundreds. With the initial values of theta all set to 1's, the hypothesis will initially evaluate to numbers nearing the thousands because living area is by far the largest feature value. Then, if you subtract the price (which are in the hundreds), you'll still end up with an error (hypothesis - target) in the thousands. As a result, each value in the vector theta will change greatly and wildly if the learning rate is too high. So we must set the learning rate to be low to ensure that theta doesn't change too greatly. If it changes too greatly, intuitively, you can visualize BGD taking huge strides and completely missing values for which theta will bring down the cost.

What if we wanted the algorithm to converge faster by using higher learning rates, that is, in the 0.001 to 0.1 range? As explained above, the variance among the ranges of feature values (thousands for living area vs single digit numbers for number of living rooms) is the source of the problem. What if we could assign a score to the living area and a score to the number of living rooms such that these scores were on the same scale? Fortunately, we can do that with a technique called **feature scaling**. One way to scale a feature would be to divide all feature values by the maximum value for that specific feature among the training set. For example, the maximum living area in our training set is 3000, so we can divide all living areas by 3000. Below, I've calculated the z-score (from statistics) of each feature value. The z-score is used to compare values in statistics, so that is why I'm using it to scale the features. The only value to not scale are the intercept terms (of 1s).

Here is a single variate implementation of [gradient descent](http://www.bogotobogo.com/python/python_numpy_batch_gradient_descent_algorithm.php).

In [63]:
def scale(X):
    all_features = []
    for i,features in enumerate(X.T[:-1]):
        mean = features.mean()
        sigma = features.std()
        scaled_features = [(f - mean) / sigma for f in features]
        all_features.append(scaled_features)
    all_features.append(X.T[-1])
    return np.array(all_features).T

coefficients = fit(scale(x), price, 0.01, do_batch_gradient_descent)
coefficients = fit(scale(x), price, 0.1, do_batch_gradient_descent)
coefficients = fit(scale(x), price, 0.2, do_batch_gradient_descent)

print
print "coefficients of the hypothesis: %s" % coefficients
for i, sample in enumerate(scale(x)):
    predicted = np.dot(sample, coefficients)
    print "predicted: $%f, actual: $%f" % (predicted, price[i])


with learning_rate = 0.01, converged after 612 iterations
with learning_rate = 0.1, converged after 79 iterations
with learning_rate = 0.2, converged after 62 iterations

coefficients of the hypothesis: [36.302200495718665, 65.394416964898667, 374.19999999999999]
predicted: $374.200000, actual: $400.000000
predicted: $342.034734, actual: $330.000000
predicted: $393.090712, actual: $369.000000
predicted: $226.894207, actual: $232.000000
predicted: $534.780347, actual: $540.000000


## Stochastic Gradient Descent
---
One thing asute readers will notice is the function `do_gradient_descent` above computes `delta` by computing the partial cost for every single training example. This means that in order to update one scalar value within theta, BGD must iterate through every training example. If your training set is small like the one used in the example above, then this isn't a problem. However, if you're Google or Facebook, and your training set contains millions or billions of samples, BGD will be slow. To account for this, **stochastic gradient descent (SGD)** updates theta once per training example. Let's implement it below. TODO: Run SGD and BGD on a large dataset.

In [69]:
def do_stochastic_gradient_descent(theta, X, Y, H, alpha):
    for (i, _) in enumerate(Y):
        new_theta = [v for v in theta]
        for j in range(len(theta)):
            new_theta[j] += alpha * partial_of_cost_wrt_j(Y, X, theta, H, i, j)
        theta = new_theta
    return theta

coefficients = fit(scale(x), price, 0.1, do_stochastic_gradient_descent)
print
print "coefficients of the hypothesis: %s" % coefficients
for i, sample in enumerate(scale(x)):
    predicted = np.dot(sample, coefficients)
    print "predicted: $%5f, actual: $%5f" % (predicted, price[i])

with learning_rate = 0.1, converged after   105 iterations

coefficients of the hypothesis: [38.379495234326747, 64.625134896357437, 374.42059941813159]
predicted: $374.420599, actual: $400.000000
predicted: $340.414763, actual: $330.000000
predicted: $394.392281, actual: $369.000000
predicted: $225.818624, actual: $232.000000
predicted: $537.056729, actual: $540.000000
